In [9]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle

# ============================================================
# 1️⃣ LOAD TRAINED BUNDLE
# ============================================================
model_path = "../models/hybrid_bundle.pkl"

with open(model_path, "rb") as f:
    bundle = pickle.load(f)

# Unpack variables from bundle
svd = bundle["svd"]
tfidf = bundle["tfidf"]
tfidf_matrix = bundle["tfidf_matrix"]
movie_indices = bundle["movie_indices"]
movies = bundle["movies"]
ratings = bundle["ratings"]

print(f"✅ Loaded trained model from {model_path}")

# ============================================================
# 2️⃣ PER-USER TRAIN/TEST SPLIT FOR EVALUATION
# ============================================================
# Filter users with at least 20 ratings for stability
ratings_filtered = ratings.groupby("userid").filter(lambda x: len(x) >= 20)

# Build per-user train/test groups (same as before)
test_ratio = 0.2
train_user_group = defaultdict(list)
test_user_group = defaultdict(list)

for uid, group in ratings_filtered.groupby("userid"):
    group = group.sample(frac=1, random_state=42)
    split = int(len(group) * (1 - test_ratio))
    train_user_group[uid] = group.iloc[:split].itertuples(index=False)
    test_user_group[uid] = group.iloc[split:].itertuples(index=False)

# ============================================================
# 3️⃣ USER PROFILE VECTOR FUNCTION
# ============================================================
def user_profile_vector(user_id):
    train_data = train_user_group[user_id]
    mids = [x.movieid for x in train_data if x.movieid in movie_indices]
    if len(mids) == 0:
        return np.zeros(tfidf_matrix.shape[1])
    idxs = [movie_indices[mid] for mid in mids]
    return np.asarray(tfidf_matrix[idxs].mean(axis=0)).flatten()

# ============================================================
# 4️⃣ HYBRID RECOMMENDER FUNCTION
# ============================================================
def recommend_top_k(user_id, k=10, alpha=0.6, n_candidates=500):
    train_movies = {x.movieid for x in train_user_group[user_id]}
    all_movies = list(set(movies.movieid) - train_movies)
    if len(all_movies) == 0:
        return pd.DataFrame(columns=['movieid','score'])

    if len(all_movies) > n_candidates:
        all_movies = np.random.choice(all_movies, n_candidates, replace=False)

    # CF scores
    cf_scores = np.array([svd.predict(user_id, mid).est for mid in all_movies])

    # Content scores
    profile = user_profile_vector(user_id)
    idxs = [movie_indices[mid] for mid in all_movies if mid in movie_indices]
    content_scores = np.zeros(len(all_movies))
    if len(idxs) > 0:
        sim = cosine_similarity(tfidf_matrix[idxs], profile.reshape(1,-1)).flatten()
        content_scores[:len(sim)] = sim

    # Normalize
    def norm(x):
        return (x - x.min()) / (x.max() - x.min() + 1e-9)

    hybrid = alpha * norm(cf_scores) + (1 - alpha) * norm(content_scores)
    return pd.DataFrame({'movieid': all_movies, 'score': hybrid}) \
             .sort_values('score', ascending=False).head(k)

# ============================================================
# 5️⃣ EVALUATION METRICS (same as your original code)
# ============================================================
# Rating prediction
predictions = [svd.predict(uid, iid) for (uid,iid,r) in 
               [(x.userid, x.movieid, x.rating) for x in ratings_filtered.itertuples(index=False)]]
y_true = [r for (_,_,r) in [(x.userid, x.movieid, x.rating) for x in ratings_filtered.itertuples(index=False)]]
y_pred = [p.est for p in predictions]

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

# Top-K metrics
K = 10
precisions, recalls, ndcgs = [], [], []
recommended_all = set()

def ndcg_at_k(recommended, actual, k=10):
    dcg = 0
    for i, mid in enumerate(recommended[:k]):
        if mid in actual:
            dcg += 1 / np.log2(i+2)
    idcg = sum(1 / np.log2(i+2) for i in range(min(len(actual), k)))
    return dcg / idcg if idcg > 0 else 0

for uid, test_data in test_user_group.items():
    actual = {x.movieid for x in test_data if x.rating >= 3.5}
    if len(actual) == 0: continue

    recs = recommend_top_k(uid, K, alpha=0.6)
    rec_set = set(recs.movieid)
    recommended_all |= rec_set

    hits = len(actual & rec_set)
    precisions.append(hits / K)
    recalls.append(hits / len(actual))
    ndcgs.append(ndcg_at_k(list(recs.movieid), actual, K))

precision_k = np.mean(precisions)
recall_k = np.mean(recalls)
ndcg_k = np.mean(ndcgs)

# Catalog coverage
catalog_coverage = len(recommended_all) / ratings_filtered.movieid.nunique()

# Diversity
def intra_list_diversity(movie_ids):
    idxs = [movie_indices[m] for m in movie_ids if m in movie_indices]
    if len(idxs) < 2:
        return 0.0
    sim = cosine_similarity(tfidf_matrix[idxs])
    n = len(idxs)
    return np.mean([1 - sim[i, j] for i in range(n) for j in range(i+1, n)])

div_scores = []
for uid in test_user_group:
    recs = recommend_top_k(uid, K, alpha=0.6)
    if len(recs) > 1:
        div_scores.append(intra_list_diversity(recs.movieid.tolist()))

avg_div = np.mean(div_scores) if len(div_scores) > 0 else 0

# Popularity-normalized hits
movie_popularity = ratings_filtered.groupby("movieid").size()
movie_popularity = movie_popularity / movie_popularity.max()
movie_popularity = movie_popularity.to_dict()
tail_threshold = np.quantile(list(movie_popularity.values()), 0.7)

def popularity_normalized_hits(recommended_ids):
    hits = 0
    norm = 0
    for mid in recommended_ids:
        p = movie_popularity.get(mid, 0)
        if p <= tail_threshold:
            hits += 1
            norm += (1 - p)
    return hits / norm if norm > 0 else 0

pop_scores = []
for uid in test_user_group:
    recs = recommend_top_k(uid, K, alpha=0.6)
    if len(recs) == 0: continue
    rec_ids = recs.movieid.tolist()
    pop_scores.append(popularity_normalized_hits(rec_ids))

avg_pop = np.mean(pop_scores) if len(pop_scores) > 0 else 0

# ============================================================
# PRINT RESULTS
# ============================================================
print("\n===== RESEARCH METRICS =====")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"Precision@10: {precision_k:.4f}")
print(f"Recall@10: {recall_k:.4f}")
print(f"NDCG@10: {ndcg_k:.4f}")
print(f"Catalog Coverage: {catalog_coverage:.4f}")
print(f"Intra-List Diversity: {avg_div:.4f}")
print(f"Popularity-Normalized Hits: {avg_pop:.4f}")


✅ Loaded trained model from ../models/hybrid_bundle.pkl

===== RESEARCH METRICS =====
RMSE: 0.7163
MAE: 0.5530
Precision@10: 0.0279
Recall@10: 0.0178
NDCG@10: 0.0367
Catalog Coverage: 0.0825
Intra-List Diversity: 0.7889
Popularity-Normalized Hits: 0.2250


   movieid     score                                        title
0     1148  0.816251  Wallace & Gromit: The Wrong Trousers (1993)
1      904  0.792507                           Rear Window (1954)
2    72226  0.785363                     Fantastic Mr. Fox (2009)
3    68954  0.756433                                    Up (2009)
4      903  0.739242                               Vertigo (1958)
5    74458  0.738890                        Shutter Island (2010)
6      923  0.736022                          Citizen Kane (1941)
7     4306  0.735048                                 Shrek (2001)
8     3114  0.733858                           Toy Story 2 (1999)
9     1089  0.732746                        Reservoir Dogs (1992)

🎯 EXPLANATIONS FOR USER 15

🎬 Wallace & Gromit: The Wrong Trousers (1993)
Genres: Animation Children Comedy Crime
Hybrid Score: 0.816
CF Score (Similar Users Liked): 3.965
Content Similarity (Matches Your Taste): 0.694
Reason: Users similar to you rated this highly + Movie